In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cross_validation import train_test_split
from tensorflow import keras

/Users/alex_y/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/alex_y/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from sklearn.utils import shuffle

def _process_image(path, dsize=(128, 128)):
    mean = [83.693283426971817, 72.796752160982763, 51.88953774401763]
    img = cv2.imread(path)[:, :, ::-1]
    # приводим все изображения к единому размеру
    img = cv2.resize(img, dsize)
    return img - mean
    

def read_train_labels(path = 'train'):
    ''' Загрузка списка файлов для обучения '''
    labels = []
    for entry in glob.glob(os.path.join(path, '*', '*.png')):
        parts = list(entry.split('/'))
        labels.append((parts[-2], entry))
    df = pd.DataFrame(data=labels, columns=('label', 'path'))
    df['label'] = df['label'].astype('category')
    return df


def load_test_data(path = 'test', dsize = (128, 128)):
    ''' Загружаем тестовые изображения '''
    paths = list(glob.glob(os.path.join(path, '*.png')))
    X = [_process_image(entry) for entry in paths]
    y = [path.split('/')[-1] for path in paths]
    return np.array(X), y
        

def _to_categorical(label, categories):
    onehot = np.zeros(len(categories), np.float32)
    onehot[categories[label]] = 1
    return onehot

def generator(df, categories, batch_size=24, dsize = (128, 128)):
    ''' Возвращает батч картинок и классов '''
    while True:
        # перемешиваем данные на очередной эпохе
        df = shuffle(df)
        # итерируем по батчам 
        for i in range(len(df) // batch_size * batch_size):
            X, y = [], []
            # итерируем по элементам в батче
            for j in range(i, i + batch_size):
                X.append(_process_image(df.values[j][1]))
                y.append(_to_categorical(df.values[j][0], categories))
            yield np.array(X), np.array(y)

## Загружаем данные для обучения

In [3]:
df = read_train_labels()

categories = dict((label, i) for i, label 
                  in enumerate(sorted(df['label'].cat.categories)))

inv_categories = dict((v, k) for k, v in categories.items())

In [4]:
df.groupby('label').count()

,path
label,
Black-grass,213
Charlock,340
Cleavers,187
Common Chickweed,561
Common wheat,171
Fat Hen,425
Loose Silky-bent,604
Maize,171
Scentless Mayweed,466


## Архитектура сверточной сети

In [5]:
### https://www.kaggle.com/miklgr500/keras-simple-model-0-97103-best-public-score

# полносвязный слой
def dense_set(inp_layer, n, activation, drop_rate=0):
    dp = keras.layers.Dropout(drop_rate)(inp_layer)
    dns = keras.layers.Dense(n)(dp)
    bn = keras.layers.BatchNormalization(axis=-1)(dns)
    act = keras.layers.Activation(activation=activation)(bn)
    return act

# сверточный слой
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = keras.layers.ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = keras.layers.Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = keras.layers.BatchNormalization(axis=3)(conv)
    act = keras.layers.LeakyReLU(1/10)(bn)
    return act

# создаем модель
''''def get_model():
    inp_img = keras.layers.Input(shape=(128, 128, 3))

    # 128
    conv1 = conv_layer(inp_img, 64, zp_flag=False)
    conv2 = conv_layer(conv1, 64, zp_flag=False)
    mp1 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # 64
    conv3 = conv_layer(mp1, 128, zp_flag=False)
    conv4 = conv_layer(conv3, 128, zp_flag=False)
    mp2 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # 32
    conv7 = conv_layer(mp2, 256, zp_flag=False)
    conv8 = conv_layer(conv7, 256, zp_flag=False)
    conv9 = conv_layer(conv8, 256, zp_flag=False)
    mp3 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # 1
    # dense layers
    flt = keras.layers.Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = keras.Model(inputs=inp_img, outputs=out)
    
    optimizer = keras.optimizers.Adam(lr=0.5 * 1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy',
                   optimizer=optimizer,
                   metrics=['accuracy'])
    
    return model''''

In [13]:
from keras.applications import Xception
model = Xception(input_shape=(128, 128, 3), include_top=False, weights='imagenet', pooling='avg')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [25]:
from keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
x = model.output
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

In [26]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.5 * 1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
              metrics=['accuracy'])

## Запускаем обучение последнего слоя

In [27]:
# откладываем часть данных для теста
train_df, test_df = train_test_split(df,
                                     test_size=0.33,
                                     random_state=123)

# мониторинг процесса обучения
lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1,
                                              epsilon=1e-5, 
                                              patience=5, 
                                              verbose=1)

checkpoint = keras.callbacks.ModelCheckpoint('plant_weights_new.h5',
                                             save_best_only=True,
                                             verbose=1)
#callbacks = [lr_reduce, msave]

# закружаем валидационные изображения в память
test_gen = generator(test_df, categories, batch_size = len(test_df))
test_X, test_y = next(test_gen)

# запускаем процесс обучения модели
batch_size = 128
train_generator = generator(train_df, categories, batch_size = batch_size)
steps_per_epoch = len(train_df) / batch_size

if os.path.exists('plant_weights_new.h5'):
    model.load_weights('plant_weights_new.h5')

model.fit_generator(train_generator,
                    steps_per_epoch,
                    epochs=30,
                    shuffle=True,
                    validation_data=(test_X, test_y),
                    callbacks=[lr_reduce, checkpoint])

Epoch 1/30
 8/21 [==========>...................] - ETA: 12:08 - loss: 2.4610 - acc: 0.1572

KeyboardInterrupt: 

## Оцениваем качество предсказания

In [ ]:
from sklearn.metrics import classification_report

actual = np.argmax(test_y, -1)
predicted = np.argmax(model.predict(test_X), -1)

print(classification_report(actual, predicted))

## Применяем обученную модель

In [ ]:
# инициализируем модель и загружаем веса
model = Xception(input_shape=(128, 128, 3), include_top=False, weights='imagenet', pooling='avg')
model.load_weights(filepath='plant_weights.h5')

# загружаем тестовые изображения
test_imgs, test_path = load_test_data()

# применяем модель и сохраняем результат
pred = np.argmax(model.predict(test_imgs, verbose=1), axis=-1)
pred_df = pd.DataFrame({'file': test_path,
                        'species': [inv_categories[p] for p in pred]})
pred_df.to_csv('submission1.csv', index=False, header=True)

# 0.79219

In [8]:
!head submission.csv

head: submission.csv: No such file or directory
